In [19]:
import csv
edges = []
with open('net.data','r') as f:
    for row in csv.reader(f.read().splitlines()):
        u, v = [i for i in row]
        edges.append((u, v))
from igraph import Graph as IGraph
g = IGraph.TupleList(edges, directed=False, vertex_name_attr='name', edge_attrs=None, weights=False)
print(g)
for p in g.vs():
    print(p['name'],p.degree())
names = g.vs['name']

IGRAPH UN-- 14 17 --
+ attr: name (v)
+ edges (vertex names):
1--2, 1--3, 2--3, 3--7, 4--5, 4--6, 5--6, 7--6, 7--8, 8--9, 9--10, 9--11,
10--11, 8--12, 12--13, 12--14, 13--14
1 2
2 2
3 3
7 3
4 2
5 2
6 3
8 3
9 3
10 2
11 2
12 3
13 2
14 2


In [54]:
#紧密中心性的算法
paths = g.get_all_shortest_paths('7')#返回的是索引号
print(paths)
cc = 0
for p in paths:
    print([names[x] for x in p])#找出真实的路径
    cc += len(p)-1
print('closeness centrality =',(len(paths)-1)/float(cc))

[[3, 2, 0], [3, 2, 1], [3, 2], [3], [3, 6, 4], [3, 6, 5], [3, 6], [3, 7], [3, 7, 8], [3, 7, 8, 9], [3, 7, 8, 10], [3, 7, 11], [3, 7, 11, 12], [3, 7, 11, 13]]
['7', '3', '1']
['7', '3', '2']
['7', '3']
['7']
['7', '6', '4']
['7', '6', '5']
['7', '6']
['7', '8']
['7', '8', '9']
['7', '8', '9', '10']
['7', '8', '9', '11']
['7', '8', '12']
['7', '8', '12', '13']
['7', '8', '12', '14']
closeness centrality = 0.48148148148148145


In [48]:
#紧密中心性函数调用
ccvs = []
for p in zip(g.vs, g.closeness()):
    ccvs.append({'name':p[0]['name'],'cc':p[1]})
sorted(ccvs, key=lambda k:k['cc'],reverse=True)

[{'name': '7', 'cc': 0.48148148148148145},
 {'name': '8', 'cc': 0.48148148148148145},
 {'name': '3', 'cc': 0.37142857142857144},
 {'name': '6', 'cc': 0.37142857142857144},
 {'name': '9', 'cc': 0.37142857142857144},
 {'name': '12', 'cc': 0.37142857142857144},
 {'name': '1', 'cc': 0.2826086956521739},
 {'name': '2', 'cc': 0.2826086956521739},
 {'name': '4', 'cc': 0.2826086956521739},
 {'name': '5', 'cc': 0.2826086956521739},
 {'name': '10', 'cc': 0.2826086956521739},
 {'name': '11', 'cc': 0.2826086956521739},
 {'name': '13', 'cc': 0.2826086956521739},
 {'name': '14', 'cc': 0.2826086956521739}]

In [69]:
#点介数中心性算法
sp = []
target = 7
for v in g.vs:
    #print(v, v['name'])
    paths = g.get_all_shortest_paths(v['name'])
    for p in paths:
        if target in p and target != p[0] and target != p[-1]:
            sp.append(p)
print(len(sp))
#去重
spbt = 0
tu = []
for x in sp:
    if set((x[0],x[-1])) not in tu:
        tu.append(set((x[0],x[-1])))
        spbt += 1
print("betweenness =", spbt)

102
betweenness = 51


In [72]:
#点介数中心性函数调用
btvs = []
for p in zip(g.vs, g.betweenness()):
    btvs.append({'name':p[0]['name'],'bt':p[1]})
sorted(btvs, key=lambda k: k['bt'], reverse=True)

[{'name': '7', 'bt': 51.0},
 {'name': '8', 'bt': 51.0},
 {'name': '3', 'bt': 22.0},
 {'name': '6', 'bt': 22.0},
 {'name': '9', 'bt': 22.0},
 {'name': '12', 'bt': 22.0},
 {'name': '1', 'bt': 0.0},
 {'name': '2', 'bt': 0.0},
 {'name': '4', 'bt': 0.0},
 {'name': '5', 'bt': 0.0},
 {'name': '10', 'bt': 0.0},
 {'name': '11', 'bt': 0.0},
 {'name': '13', 'bt': 0.0},
 {'name': '14', 'bt': 0.0}]

In [74]:
#pagerank算法
pg = g.pagerank()
pgvs = []
for p in zip(g.vs, pg):
    pgvs.append({'name':p[0]['name'],'pg':p[1]})
sorted(pgvs,key=lambda k: k['pg'], reverse=True)

[{'name': '3', 'pg': 0.08621177802944507},
 {'name': '6', 'pg': 0.08621177802944507},
 {'name': '9', 'pg': 0.08621177802944506},
 {'name': '12', 'pg': 0.08621177802944506},
 {'name': '7', 'pg': 0.08311761850833196},
 {'name': '8', 'pg': 0.08311761850833196},
 {'name': '2', 'pg': 0.06111470635819448},
 {'name': '5', 'pg': 0.06111470635819448},
 {'name': '14', 'pg': 0.06111470635819448},
 {'name': '1', 'pg': 0.06111470635819447},
 {'name': '4', 'pg': 0.06111470635819447},
 {'name': '13', 'pg': 0.06111470635819447},
 {'name': '10', 'pg': 0.06111470635819446},
 {'name': '11', 'pg': 0.06111470635819446}]

In [80]:
#边介数中心性（社区发现算法相关）
#也是GN算法
btes = []
for p in zip(g.es, g.edge_betweenness()):
    e = p[0].tuple
    btes.append({'edge':(names[e[0]],names[e[1]]),'bt':p[1]})
sorted(btes, key=lambda k:k['bt'], reverse=True)

[{'edge': ('7', '8'), 'bt': 49.0},
 {'edge': ('3', '7'), 'bt': 33.0},
 {'edge': ('7', '6'), 'bt': 33.0},
 {'edge': ('8', '9'), 'bt': 33.0},
 {'edge': ('8', '12'), 'bt': 33.0},
 {'edge': ('1', '3'), 'bt': 12.0},
 {'edge': ('2', '3'), 'bt': 12.0},
 {'edge': ('4', '6'), 'bt': 12.0},
 {'edge': ('5', '6'), 'bt': 12.0},
 {'edge': ('9', '10'), 'bt': 12.0},
 {'edge': ('9', '11'), 'bt': 12.0},
 {'edge': ('12', '13'), 'bt': 12.0},
 {'edge': ('12', '14'), 'bt': 12.0},
 {'edge': ('1', '2'), 'bt': 1.0},
 {'edge': ('4', '5'), 'bt': 1.0},
 {'edge': ('10', '11'), 'bt': 1.0},
 {'edge': ('13', '14'), 'bt': 1.0}]

In [84]:
#社区发现算法的函数调用
communities = g.community_edge_betweenness(directed=False, weights=None)
print(communities)
print(g.vs['name'])

Dendrogram, 14 elements, 13 merges

0 1 2 3 4 5 6 7 11 12 13 8 9 10
| | | | | | | | |  |  |  | | |
| | | | | | | | |  `--'  | | |
| | | | | | | | |   |    | | |
| | | | | | | | `---'    | `-'
| | | | | | | |   |      |  | 
| | | | | `-' |   |      `--' 
| | | | |  |  |   |       |   
| `-' | `--'  |   |       |   
|  |  |  |    |   |       |   
`--'  `--'    `---'       |   
 |     |        |         |   
 `-----'        `---------'   
    |                |        
    `----------------'
['1', '2', '3', '7', '4', '5', '6', '8', '9', '10', '11', '12', '13', '14']
